In [1]:
from chemprop.nn.message_passing.base import BondMessagePassing, AtomMessagePassing

Example dataloader to make inputs to the message passing layer. See the dataloader notebook for more details.

In [2]:
import numpy as np
from chemprop.data import MoleculeDatapoint, MoleculeDataset, build_dataloader

smis = ["C" * i for i in range(1, 4)]
ys = np.random.rand(len(smis), 1)
dataset = MoleculeDataset([MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)])
dataloader = build_dataloader(dataset)

### Message Passing

There are two message passing schemes. Chemprop prefers a D-MPNN scheme (`BondMessagePassing`) where messages are passed between directed edges (bonds) rather than between nodes (atoms) as would be done in a traditional MPNN (`AtomMessagePassing`).

In [3]:
mp = AtomMessagePassing()
mp = BondMessagePassing()

### Input dimensions

By default, the bond message passing layer's input dimension is the sum of atom and bond features from the default atom and bond featurizers (see the featurizer notebook for more details). If you use a custom featurizer, the message passing layer needs to be told when created.

Also note that an atom message passing's default input dimension is the length of the atom features from the default atom featurizer.

In [4]:
from chemprop.featurizers import SimpleMoleculeMolGraphFeaturizer

n_atom_features, n_bond_features = SimpleMoleculeMolGraphFeaturizer().shape
(n_atom_features + n_bond_features) == mp.W_i.in_features

True

In [5]:
from chemprop.featurizers import MultiHotAtomFeaturizer

n_bonds = 1
n_extra_bond_features = 12
extra_bond_features = np.random.rand(n_bonds, n_extra_bond_features)
featurizer = SimpleMoleculeMolGraphFeaturizer(
    atom_featurizer=MultiHotAtomFeaturizer.organic(), extra_bond_fdim=n_extra_bond_features
)
dataset = MoleculeDataset(
    data=[MoleculeDatapoint.from_smi(smi, y, E_f=extra_bond_features) for smi, y in zip(smis, ys)],
    featurizer=featurizer,
)

mp = BondMessagePassing(d_v=featurizer.atom_fdim, d_e=featurizer.bond_fdim)

If extra atom descriptors are used, the message passing layer also needs to be told. A separate weight matrix is created and optimized to transform the concatenated hidden representation and extra descriptors back to the hidden dimension after message passing is complete. 

In [6]:
n_atoms = 2
n_extra_atom_descriptors = 28
extra_atom_descriptors = np.random.rand(n_atoms, n_extra_atom_descriptors)
dataset = MoleculeDataset(
    [MoleculeDatapoint.from_smi(smi, y, V_d=extra_bond_features) for smi, y in zip(smis, ys)]
)

mp = BondMessagePassing(d_vd=n_extra_atom_descriptors)

### Customization

The following hyperparameters of the message passing layer are customizable:
 - the hidden dimension during message passing, default: 300
 - whether a bias term used, default: False
 - the number of message passing iterations, default: 3
 - whether to pass messages on undirected edges, default: False
 - the dropout probability, default: 0.0 (i.e. no dropout)
 - which activation function, default: ReLU

In [7]:
mp = BondMessagePassing(
    d_h=600, bias=True, depth=5, undirected=True, dropout=0.5, activation="tanh"
)

The output of message passing is a torch tensor of shape # of atoms in batch x length of hidden representation.

In [8]:
batch_molgraph, extra_atom_descriptors, *_ = next(iter(dataloader))
hidden_atom_representations = mp(batch_molgraph, extra_atom_descriptors)
hidden_atom_representations.shape

torch.Size([6, 600])

### Scaling

Scaling transforms for extra atom features, bond features and atom descriptors are given to the message passing layer to be used during inference. See the scaling notebook for more details.

### Computations

See the source code or [Chemprop paper](https://pubs.acs.org/doi/10.1021/acs.jcim.3c01250) for details of the actual calculations done in the message passing layer. 